In [1]:
import os


In [2]:
from typing import Annotated, List, Tuple, Union

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool

tavily_tool = TavilySearchResults(max_results=5)

# This executes code locally, which can be unsafe
python_repl_tool = PythonREPLTool()

In [3]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [4]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser

members = ["Researcher","Product Manager", "Startup Consultant", "Business Strategist", "Financial Analyst", "Software Engineer", "Philisopher"]
system_prompt = (
    "You are a Leader of a panel of experts tasked with managing a conversation between the"
    " following experts:  {members}. Given the following user request,"
    " respond with the expert to act next. Each expert will perform an"
    " analysis on how to succeed and respond with their analysis and status. When finished,"
    " respond with FINISH."
    " When finished you (the Leader) will provide a summary of the conversation."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next expert.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-4-turbo")

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [6]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


research_agent = create_agent(llm, [tavily_tool], "You are a web researcher.")
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# Product Manager
product_manager_agent = create_agent(llm, [tavily_tool], "You are a Product Manager.")
product_manager_node = functools.partial(agent_node, agent=product_manager_agent, name="Product Manager")

# Startup Consultant
startup_consultant_agent = create_agent(llm, [tavily_tool], "You are a Startup Consultant.")
startup_consultant_node = functools.partial(agent_node, agent=startup_consultant_agent, name="Startup Consultant")

# Business Strategist
business_strategist_agent = create_agent(llm, [tavily_tool], "You are a Business Strategist.")
business_strategist_node = functools.partial(agent_node, agent=business_strategist_agent, name="Business Strategist")

# Financial Analyst
financial_analyst_agent = create_agent(llm, [tavily_tool], "You are a Financial Analyst.")
financial_analyst_node = functools.partial(agent_node, agent=financial_analyst_agent, name="Financial Analyst")

# Software Engineer
software_engineer_agent = create_agent(llm, [tavily_tool], "You are a Software Engineer.")
software_engineer_node = functools.partial(agent_node, agent=software_engineer_agent, name="Software Engineer")

# Philisopher
philisopher_agent = create_agent(llm, [tavily_tool], "You are a Philisopher.")
philisopher_node = functools.partial(agent_node, agent=philisopher_agent, name="Philisopher")


workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Product Manager", product_manager_node)
workflow.add_node("Startup Consultant", startup_consultant_node)
workflow.add_node("Business Strategist", business_strategist_node)
workflow.add_node("Financial Analyst", financial_analyst_node)
workflow.add_node("Software Engineer", software_engineer_node)
workflow.add_node("Philisopher", philisopher_node)
workflow.add_node("supervisor", supervisor_chain)

In [7]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("supervisor")

graph = workflow.compile()

In [8]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="I want to to start a new business scraping and selling product price data. Can you help me develop this idea?")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Startup Consultant'}}
----
{'Startup Consultant': {'messages': [HumanMessage(content="Starting a business that involves scraping and selling product price data involves several important steps and considerations. Here’s a breakdown of the key aspects you'll need to consider:\n\n### 1. **Legal and Compliance**\n- **Data Privacy Laws**: Understand the legal implications of scraping data, especially personal data or data protected by copyright laws (e.g., GDPR in Europe, CCPA in California). \n- **Terms of Service**: Review the terms of service of websites from which you plan to scrape data to ensure you are not violating any terms.\n\n### 2. **Business Model**\n- **Target Market**: Identify who your customers will be. This could be businesses in competitive industries, market analysts, or even academic researchers.\n- **Value Proposition**: Determine how your data will offer value. This might be through providing real-time pricing updates, historical price trends

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'messages[2].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[2].name', 'code': 'invalid_value'}}

In [ ]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Write a brief research report on pikas.")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

In [ ]:
print('# Research Report on Pikas\n\nPikas are small, short-legged, and virtually tailless mammals with an egg-shaped body that can be found in the mountains of western North America and much of Asia. They are the only living members of the family Ochotonidae, with the genus Ochotona being the sole representative. Despite their apparent resemblance to rodents, pikas are part of the lagomorph group, which also includes hares and rabbits.\n\n## Description and Behavior\n\nPikas typically have soft, long, and thick brownish or reddish fur. They weigh between 4.5 and 7.1 ounces (125 and 200 grams) and measure about 6 inches (15 cm) in length. Unlike many other mammals that inhabit cold regions, pikas do not hibernate. Instead, they remain active throughout the winter, moving through tunnels under rocks and snow. In summer and autumn, they engage in a behavior known as "haying," where they harvest vegetation and store it in protected places to serve as a food reserve during the winter months.\n\nPikas are particularly adept at surviving in harsh alpine environments, the windswept areas above the tree line. They can endure the entire span of their lives in these conditions, making them one of the few mammals in the lower 48 states of the US capable of such resilience.\n\n## Distribution and Habitat\n\nPikas inhabit rocky, mountainous areas, with some species living in burrows. Eurasian pikas often live in family groups and share duties such as gathering food and keeping watch. The range of Ochotona was larger in the past, with both extinct and extant species having inhabited Western Europe and Eastern North America, areas that are currently free of pikas.\n\nThe American pika (Ochotona princeps) is found throughout the mountains of western North America, from central British Columbia and Alberta in Canada down to various US states. It is one of only two pika species that inhabit North America, the other being the collared pika (O. collaris).\n\n## Conservation and Threats\n\nPikas are sensitive to high temperatures due to their preference for cooler mountain regions. They are considered early indicators of global warming, as they have been observed moving to higher elevations in search of suitable and cooler habitats. Recent studies indicate that some populations are declining due to various factors, most notably climate change, with warmer summer and winter temperatures, changes in precipitation, and reduced winter snowpack among the contributing factors.\n\nAlthough the American pika was considered for listing under the US Endangered Species Act, it was ultimately not added. Nonetheless, widespread extirpations and range retractions at lower elevations have been documented, and these patterns are often attributed to the effects of climate change.\n\n## Interaction with Humans\n\nWhile most pikas live far from human habitation, some species, particularly burrowing pikas on the Tibetan plateau, have been considered pests. There, they are thought to reduce forage for domestic livestock and to cause damage to grasslands.\n\n## Conclusion\n\nPikas are fascinating creatures that play an essential role in their alpine ecosystems. Their unique adaptations allow them to thrive in some of the most inhospitable environments on Earth. However, their sensitivity to temperature changes makes them vulnerable to the impacts of climate change, underscoring the need for continued research and conservation efforts to ensure their survival.\n\n---\n\nReferences:\n- National Wildlife Federation: [American Pika](https://www.nwf.org/Educational-Resources/Wildlife-Guide/Mammals/American-Pika)\n- Britannica: [Pika](https://www.britannica.com/summary/pika)\n- Wikipedia: [Pika](https://en.wikipedia.org/wiki/Pika)\n- Wikipedia: [American pika](https://en.wikipedia.org/wiki/American_pika)\n- Britannica: [Pika (genus Ochotona)](https://www.britannica.com/animal/pika)')